# Persona-Hub による事後学習データ合成

Susumu Ota  2025-02-02

本ハンズオンでは、言語モデルを使って合成データを生成する方法を紹介します。

まず、簡単に合成データ生成の概要を説明します。次に、[Persona-Hub](https://arxiv.org/abs/2406.20094) 手法の解説、[FinePersonas](https://huggingface.co/datasets/argilla/FinePersonas-v0.1) データセットの紹介、最後に実際にペルソナデータを使ってマルチターン事後学習データを合成する方法を説明します。

## 合成データ生成の概要

### 合成データの重要性

本ハンズオンにおける合成データとは、"**人間によって直接生成されたデータではなく、モデルやアルゴリズムによって生成されたデータ**"を指します。

合成データの重要性については、以下のような報告があります。

- [畠山先生の 松尾研LLM開発プロジェクト “Tanuki-8x8B” 開発成果報告会 Vol.1 発表資料](https://www.docswell.com/s/matsuo-lab_llm/51R2L4-2024-9-10-Tanuki%E9%96%8B%E7%99%BA%E5%A0%B1%E5%91%8A%E4%BC%9A-vol1#p54)より
  - Web データで継続事前学習しても JMT-Bench のスコアは横ばい
  - 合成データの投入でスコアが向上
  - 数学・論理推論・コードが難しい<br />
<img src="https://github.com/user-attachments/assets/2eb9d26b-cddc-4c8f-b8a1-be9e6d1d3c05" width="800px">

- [合成データの多様さと言語モデルの学習への影響を調べたプレプリント](https://arxiv.org/abs/2410.15226)より
  - 合成データの多様さと、学習後のモデルの性能に正の相関がある (多様な合成データを使うほど性能が向上)
  - モデルサイズが大きいほど、合成データの多様さが性能に与える影響が大きい<br />
<img src="https://github.com/user-attachments/assets/ccdc7e34-fdfd-4810-9ef4-8c2f2145bb86" width="800px">

### 合成データ作成の難しさ

一般に、データ生成タスクにおいて、データの**質**と**多様さ**についてはトレードオフの関係があります。例えば、人手でデータを作成する場合は、予算一定では品質と多様さのどちらかを犠牲にせざるを得ません。

一方、言語モデルを使った合成データ生成の場合も、データの**質**と**多様さ**については同様にトレードオフがありますが、以下のような特徴があります。

<img src="https://github.com/user-attachments/assets/63e7a21e-ad8f-40a2-b22e-86eddef6e623" width="400px">

- 品質は合成に使う言語モデルの性能が上がれば向上 (Scaling Laws に乗っかることが可能) (橙色の上矢印)
- 多様さを向上させることが難しい (緑色の右矢印)

したがって、合成データの多様さを向上させるためには、何らかのヒント・種を言語モデルに与えた上で合成データを出力する必要があります。

## 本ハンズオンの目的

本ハンズオンの目的は、言語モデルを使った合成データの作成において、多様さを向上させるための方法を紹介することです。特に、[Persona-Hub](https://arxiv.org/abs/2406.20094) という手法を使って、事後学習用のマルチターン対話データを生成する方法を紹介します。

多様な合成データを生成する方法として以下のような方法が提案されています([Persona-Hub のテクニカルレポート](https://arxiv.org/abs/2406.20094)より)。

- インスタンス駆動
  - 例: Wikipedia の記事から Q&A を生成
- キーポイント駆動
  - 例: 数学の学習指導要領に含まれる用語 (e.g. `三角関数`) から数学の問題を生成
- ペルソナ駆動
  - 例: ペルソナ (e.g. `運送会社のドライバー`) から数学の問題を生成

本ハンズオンでは3つ目の**ペルソナ駆動による合成データ生成**を中心に説明します。

なお、上記と直交する方法として、多様な合成データを生成するためにサンプリングを行うという方法があります。例えば、言語モデルで合成データを出力する際に、温度パラメータを高めに設定することで、多様な出力を得ることができます。しかし、温度を上げすぎるとハルシネーションや文が破綻する可能性が高まるため、品質と多様さのトレードオフが生じます。したがって、サンプリングのみで得られる多様さは限定的です。

## 準備

ここからは、合成データ生成に必要な推論用言語モデルの準備を行います。言語モデルの推論を行うために、既存の推論 API を利用する方法と、Colab の GPU やローカルの GPU を使って推論を行う方法のどちらかを選択することができます。

### トークンと API キーの設定

本ハンズオンで必要となるトークンや API キーを設定します。

#### Hugging Face のトークンの取得

Hugging Face からデータセットやモデルをダウンロードするためにトークンが必要となります。サインアップしてトークンを取得し `HF_TOKEN` という環境変数に設定してください。

Hugging Face にサインアップしてトークンを取得する方法は、[こちら](https://zenn.dev/protoout/articles/73-hugging-face-setup)の記事を参照してください。生成する際の `role` は `read` で十分です。

#### (オプション) 言語モデルサービスの API キーの取得

Note: **Colab の GPU やローカルの GPU を使って推論する場合は、この設定は不要**です。

推論 API の実行は [LiteLLM](https://docs.litellm.ai/) というモジュール経由で行います。LiteLLM の [Providers](https://docs.litellm.ai/docs/providers) ページを参照して、対応しているサービス一覧と環境変数名を確認してください。

代表的な API キーの取得方法は以下です。
- `OPENAI_API_KEY`
  - OpenAI の API キーの取得方法は、[こちら](https://qiita.com/kurata04/items/a10bdc44cc0d1e62dad3)の記事を参照してください。
  - 2025-02-02現在、無料枠はなく、利用開始時に $5 のクレジットが必要となります。
- `NVIDIA_NIM_API_KEY`
  - NVIDIA NIM の API キーの取得方法は、[こちら](https://zenn.dev/connectome/articles/eb9848241c5115)の記事のAPIキーを取得する部分までを参照してください。
  - 2025-02-02現在、NVIDIA NIM の API は **1000 リクエストまでは無料**で利用できます。

#### トークンと API キーをシークレットマネージャーに保存

この Notebook をローカル環境等の安全な環境で実行する場合は、OS の環境変数でトークンや API キーを保存してください。

Colab 等のクラウド環境で実行する場合は、シークレットマネージャーに保存してください。**ソースコード中に API キーやトークンを直接書くとセキュリティ上のリスクが高まります**。

Colab でのシークレットマネージャーによる設定方法は以下のようなコードを実行してください。詳細は[こちら](https://note.com/npaka/n/n79bb63e17685)の記事を参照してください。

In [1]:
if str(get_ipython()).startswith("<google.colab"):  # if this notebook is running in Google Colab  # type: ignore
    import os
    from google.colab import userdata  # type: ignore

    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
    # os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    # os.environ["NVIDIA_NIM_API_KEY"] = userdata.get("NVIDIA_NIM_API_KEY")

### モジュールのインストール

`litellm` と `datasets` をインストールしてください。ローカルのGPUを使って推論する場合は `vllm` もインストールしてください(API だけを使う場合は不要です)。

In [2]:
# %pip install litellm
# %pip install datasets

# %pip install vllm

### モジュールのインポート

In [3]:
from abc import ABC, abstractmethod
import json
from logging import DEBUG, INFO, StreamHandler, getLogger  # noqa: F401
import pprint
import random
import re


from datasets import load_dataset
from litellm import batch_completion

try:
    from vllm import LLM, SamplingParams  # type: ignore
except ImportError:
    print("No vllm module found. You can only use the LiteLLM.")

/Users/ota/Documents/python/synthetic-data-hands-on/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ota/Documents/python/synthetic-data-hands-on/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


No vllm module found. You can only use the LiteLLM.


### ログの設定

In [4]:
logging_level = DEBUG
# logging_level = INFO  # uncomment if you want to see less output
logger = getLogger(__name__)
logger.setLevel(logging_level)
handler = StreamHandler()
handler.setLevel(logging_level)
logger.addHandler(handler)

pp = pprint.PrettyPrinter(width=100)

### 言語モデルの設定

API による推論とローカル GPU を使った推論の両方を統一してコードを書くために、`LanguageModel` という抽象クラスを作り、サブクラスで個別の処理を実装します。

推論 API は `LiteLLM` 経由で利用します。

Colab の GPU やローカルの GPU を使って推論には `vLLM` を使います。`LiteLLM` 経由で `vLLM` を使うことも出来ますが、`LiteLLM` では `dtype` の設定が出来ない(これが出来ないと Colab T4 で動作しない)ため `vLLM` を直接使うことにしました。

`LanguageModel` のメソッドの役割は以下の通りです。

- `__init__`: 言語モデルの初期化。推論用のパラメータ(`temperature`等)も設定。
- `__call__`: 推論の実行。入力は OpenAI messages 形式 (e.g. `[{"role": "user", "content": "Hello!"}]`) のリスト。出力は文字列のリスト。

In [5]:
class LanguageModel(ABC):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None):
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.seed = seed
        logger.debug(f"model: {model}, temperature: {temperature}, max_tokens: {max_tokens}, seed: {seed}")

    @abstractmethod
    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        pass


class LiteLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None):
        super().__init__(model, temperature, max_tokens, seed)

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        contents = [
            response.choices[0].message.content or ""
            for response in batch_completion(
                model=self.model,
                messages=messages_batch,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
            )
        ]
        assert len(contents) == len(messages_batch)
        return contents


class VLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, dtype="auto", stop=None):
        super().__init__(model, temperature, max_tokens, seed)
        self.dtype = dtype
        self.stop = stop
        self.vllm = LLM(model, dtype=dtype)  # dtype must be "half" to run on Colab T4
        self.tokenizer = self.vllm.get_tokenizer()

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        sampling_params = SamplingParams(
            temperature=self.temperature, max_tokens=self.max_tokens, seed=self.seed, stop=self.stop
        )
        prompts = [
            self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            for messages in messages_batch
        ]
        outputs = self.vllm.generate(prompts, sampling_params=sampling_params, use_tqdm=False)
        contents = [o.outputs[0].text for o in outputs]
        assert len(contents) == len(messages_batch)
        return contents

### 言語モデルの初期化

推論 API を使う場合は `準備` の節を参考に API キーを設定してください。環境変数名やモデル名は、[LiteLLM の Providers ページ](https://docs.litellm.ai/docs/providers)を参照してください。性能と価格のバランスを考慮すると、本ハンズオンで行う程度の内容であれば、`gpt-4o-mini` が適切かもしれません。

Colab T4 GPU を使って推論する場合は `vLLM` を使います。手元でテストした限りでは、T4 では、

- 3B 前後の量子化していないモデル
- 10B 前後の量子化したモデル

が動作可能でした。

本ハンズオンの内容を `google/gemma-2-9b-it` の非公式量子化版の `marcsun13/gemma-2-9b-it-GPTQ` で動作確認をしましたが、合成データの質は `gpt-4o-mini` と比べて**大幅に劣ります**。可能であれば、A100 等の GPU を使い、 `cyberagent/calm3-22b-chat` やそれと同等以上の性能を持つモデルを使うことをお勧めします。サイズの小さいモデルや品質の低いモデルでは、合成データ生成がうまくいかない場合があります。個人的な印象ですが、現状では 10B 以下の日本語モデルでは、合成データ生成は難しいかもしれません。

また、モデル・API のライセンスや利用規約等を確認して、**合成データを利用する際の制限事項等を各自で確認してください**。

### 推論 API を使う場合の注意点

一般に API の推論速度は遅いので、予備実験を API で行い、本実験は vLLM でローカル GPU で行うという使い方が現実的かもしれません。

また、API で大量のデータを生成する場合は、バッチ処理のオプションが用意されていればそれを利用することと、利用制限(rate limit等)の範囲内でリクエストの並列化を検討してください。LiteLLM は内部でマルチスレッドで並列化されていますが、バッチサイズ大きめで使うと API サービス側の rate limit を超える可能性があります。

### 言語モデルの作成

以下のどれかのコメントを外して言語モデルを初期化します。

In [6]:
llm = LiteLLMModel("gpt-4o-mini", temperature=0.7, max_tokens=512, seed=0)  # OPENAI_API_KEY
# llm = LiteLLMModel("nvidia_nim/nvidia/nemotron-4-340b-instruct", temperature=0.7, max_tokens=512, seed=None)  # NVIDIA_NIM_API_KEY
# llm = LiteLLMModel("nvidia_nim/nvidia/llama-3.1-nemotron-70b-instruct", temperature=0.7, max_tokens=512, seed=None)  # NVIDIA_NIM_API_KEY
# llm = LiteLLMModel("deepinfra/nvidia/Llama-3.1-Nemotron-70B-Instruct", temperature=0.7, max_tokens=512, seed=0)  # DEEPINFRA_API_KEY
# llm = VLLMModel("hpprc/gemma-2-2b-jpn-it", temperature=0.7, max_tokens=512, seed=0, stop=["<end_of_turn>"], dtype="half")  # for Colab T4
# llm = VLLMModel("marcsun13/gemma-2-9b-it-GPTQ", temperature=0.7, max_tokens=512, seed=0)  # for Colab T4
# llm = VLLMModel("cyberagent/calm3-22b-chat", temperature=0.7, max_tokens=512, seed=0)  # for A100?

llm

model: gpt-4o-mini, temperature: 0.7, max_tokens: 512, seed: 0


### 言語モデルの動作確認

ここまで設定できれば以下の推論が動作するはずです。もしエラーが出る場合は、言語モデルの設定を見直してください。

In [7]:
llm([[{"role": "user", "content": "Hello?"}]])

['Hello! How can I assist you today?']

ここまでで言語モデルによる推論環境を構築することできました。

次に、ペルソナ駆動による合成データ生成の方法について説明します。

## Persona-Hub による合成データ生成

### ペルソナとは

[FinePersonasのREADME](https://huggingface.co/datasets/argilla/FinePersonas-v0.1) では、ペルソナについて以下のように説明されています。

> - 個人の特徴、背景、目標を詳細に記述したもので、多様なアイデンティティと経験を反映するようにデザインされている
> - ペルソナの例
>   - A network engineer with a focus on routing protocols and preparing for Cisco certification exams, particularly CCNA.
>   - ルーティング・プロトコルに興味があり、シスコの認定試験(特に CCNA)の準備をしているネットワーク・エンジニア
> - 生成するコンテンツに、特定の専門知識・キャリアパス・個人的な興味を導入し、より繊細でターゲットを絞ったコンテンツが生成可能

### Persona-Hub とは

[Persona-Hub](https://arxiv.org/abs/2406.20094) は、Tencent AI Lab が提案したペルソナ駆動型データ合成手法で、大規模言語モデル内の様々な視点を活用して多様な合成データを作成することができます。

<img src="https://github.com/user-attachments/assets/37ce038a-7702-4398-838d-8c504ac1da07" width="800px">

Persona-Hub 手法の概要は以下の通りです。

- 背景: 既存の合成データ⽣成⼿法(インスタンス駆動・キーポイント駆動)では合成データの多様さをスケールアップすることが困難
- 目的: ⼤規模なペルソナデータセットを作成し、それを使ってスケーラブルな合成データを⽣成する(ペルソナ駆動)
- 方法
  - Web ページのテキストからペルソナを抽出 (上図の`Compress`部分)
    - Text-to-Persona
    - Persona-to-Persona
  - ペルソナを使って合成データを生成 (上図の`Decompress`部分、下図)
    - Create **a math problem** with **a moving company driver**
    - Create **a math problem** with **a chemical kinetics researcher**
    - Create **a math problem** with **a musician interested in audio processing**
- 結果: 10億件のペルソナデータセットを作成 (ただし今のところ非公開)

<img src="https://github.com/user-attachments/assets/344b011d-b9f8-4ac3-a79a-198e3862b3cf" width="800px">

### Web ページのテキストからペルソナを抽出

まず、Persona-Hub 手法を理解するために、Web ページのテキストからペルソナを抽出する方法を説明します(上図の`Compress`部分)。その後、抽出したペルソナを使って合成データを生成する方法を説明します(上図の`Decompress`部分)。

ここでは、実際に [FineWeb-Edu](https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu) という教育関連の品質の高い Web ページを集めたデータセットからペルソナを抽出します。

まずデータセットを読み込みます。データセット全体を読み込むと時間がかかるので、ここでは `streaming=True` を指定して一部分だけ読み込みます。

In [8]:
fineweb_edu = load_dataset(
    "HuggingFaceFW/fineweb-edu", name="CC-MAIN-2024-51", split="train", cache_dir="cache", streaming=True
)

fineweb_edu

IterableDataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
    num_shards: 50
})

今回は実験のため、データセットの先頭の10件を取り出して `web_pages` というリストに格納します。

In [9]:
web_pages = [d["text"] for d in fineweb_edu.take(10)]
[p[:100] for p in web_pages]

['This project is solving the Asteroid Watchers challenge. Description\nAROs are essentially created by',
 'Life of a Sand Grain\nTHE LIFE OF A SAND GRAIN by Carl Bowser (Sept. 2018)\nThey surround you almost a',
 'The internet of things, a system of interrelated computing devices and machines that can transfer da',
 'An archive photo of an Egyptian mummy - Reuters\nBy Tom Perry\nCAIRO, Jan 15 (Reuters) - Archaeologist',
 'The faith of the Christ-God is a living paradox in the Asiatic world. Christianity has long survived',
 'Power wound resistance is a two terminal electronic component made of\nresistance material, which has',
 'Pravda No. 50, March 1, 1913 |\nPublished according to |\nFrom V. I. Lenin, Collected Works, 4th Engli',
 '1. 03. Friend B: I look washed out. Publications Publications such as books, magazines, newspapers, ',
 'Aerospace & Electronic Techniques Society\nUntil 1950, this field was called “radio expertise” as a e',
 'The Dawn of the Artificial Kidney\nArtificia

上記のように教育分野の比較的品質の高い Web ページを集めたデータセットです。特に科学技術分野のペルソナを抽出できそうです。

#### Text-to-Persona

Persona-Hub では、ペルソナを抽出する手法として `Text-to-Persona` と `Persona-to-Persona` が提案されています。ここでは Text-to-Persona 手法によって、Web ページのテキストからペルソナを抽出します。

効果的にペルソナを抽出するポイントは `このテキストを書きそうな人物` や `このテキストに興味がありそうな人物` を言語モデルに予測させることです。

以下のようなプロンプトを実行します。

Note: `system` ロールがサポートされていない言語モデルを使う場合は `user` ロールにシステムプロンプトの内容を含めてください。

In [10]:
def text_to_persona(llm: LanguageModel, text: str) -> str:
    SYSTEM_PROMPT = """\
You are an expert in analyzing the text content and assigning finding the general type of persona that could be associated with such a way of expressing. Please use one or two sentences for the definition, but try to make it as fine-grained if input texts involve many detailed elements. The persona definition must go straight to the point, be assertive. The following are starts of persona definitions:
A machine learning researcher...
A pedriatric nurse whose...
An urban planner focused on...
"""

    USER_PROMPT = """\
What is the likely profession, interest, or role of the person who would write or be interested in this text?

## Text
{text}

Note:
1. Your response should always start with "ペルソナ:".
2. 日本語で回答してください。
"""
    persona = llm([[
        # {"role": "system", "content": SYSTEM_PROMPT},  # if llm supports system prompt
        # {"role": "user", "content": USER_PROMPT.format(text=text)},
        {"role": "user", "content": SYSTEM_PROMPT + "\n\n## Task\n" + USER_PROMPT.format(text=text)},  # if llm does not support system prompt
    ]])[0]
    logger.debug(f"persona: {persona}")

    return re.sub(r"^ペルソナ[:：]", "", persona).strip()

この関数を使って FineWeb-Edu データからペルソナを抽出します。

In [11]:
personas = [text_to_persona(llm, web_page) for web_page in web_pages]
personas

persona: ペルソナ: オープンソースハードウェアとソフトウェアに関心を持つ天文愛好家であり、Arduinoやスマートフォンを用いた技術的なプロジェクトを推進するエンジニア。
persona: ペルソナ: 地質学者であり、特に砂や鉱物の成り立ちや成熟度に関心を持つ研究者。彼らは自然環境の変遷や地球の歴史を理解するために、砂のサンプルを収集し分析することを楽しむ。
persona: ペルソナ: IoTおよびAI技術の専門家であり、データ分析やシステム設計に関心を持つエンジニアまたは研究者。
persona: ペルソナ: 古代エジプトの考古学者で、歴史的な発見や文化遺産の保護に情熱を持ち、専門的な知識を駆使して研究を行っている。
persona: ペルソナ: このテキストは、宗教的歴史や文化的相互作用に関心を持つ宗教学者または文化人類学者が書いたものであり、特にキリスト教とアジアの宗教との関係に焦点を当てた分析を行う専門家の視点を反映している。
persona: ペルソナ: 電子工学の研究者で、回路設計や抵抗器の特性に関心を持つ技術者。
persona: ペルソナ: マルクス主義の理論を深く研究し、社会主義社会の建設における労働者階級の役割に関心を持つ政治理論家や歴史家。
persona: ペルソナ: 印刷メディアや広告戦略に関心を持つマーケティング専門家であり、特に伝統的なメディアの進化やその効果的な活用方法に注力しているプロフェッショナル。
persona: ペルソナ: 航空宇宙および電子工学の専門家であり、電子機器の設計や技術的原理に精通したエンジニアで、最新の技術トレンドや産業動向に関心を持つ人物。
persona: ペルソナ: 人工腎臓技術の研究者であり、医療技術の進歩に情熱を持つ科学者。患者の生活の質向上を目指し、革新的な治療法の開発に取り組んでいる。


['オープンソースハードウェアとソフトウェアに関心を持つ天文愛好家であり、Arduinoやスマートフォンを用いた技術的なプロジェクトを推進するエンジニア。',
 '地質学者であり、特に砂や鉱物の成り立ちや成熟度に関心を持つ研究者。彼らは自然環境の変遷や地球の歴史を理解するために、砂のサンプルを収集し分析することを楽しむ。',
 'IoTおよびAI技術の専門家であり、データ分析やシステム設計に関心を持つエンジニアまたは研究者。',
 '古代エジプトの考古学者で、歴史的な発見や文化遺産の保護に情熱を持ち、専門的な知識を駆使して研究を行っている。',
 'このテキストは、宗教的歴史や文化的相互作用に関心を持つ宗教学者または文化人類学者が書いたものであり、特にキリスト教とアジアの宗教との関係に焦点を当てた分析を行う専門家の視点を反映している。',
 '電子工学の研究者で、回路設計や抵抗器の特性に関心を持つ技術者。',
 'マルクス主義の理論を深く研究し、社会主義社会の建設における労働者階級の役割に関心を持つ政治理論家や歴史家。',
 '印刷メディアや広告戦略に関心を持つマーケティング専門家であり、特に伝統的なメディアの進化やその効果的な活用方法に注力しているプロフェッショナル。',
 '航空宇宙および電子工学の専門家であり、電子機器の設計や技術的原理に精通したエンジニアで、最新の技術トレンドや産業動向に関心を持つ人物。',
 '人工腎臓技術の研究者であり、医療技術の進歩に情熱を持つ科学者。患者の生活の質向上を目指し、革新的な治療法の開発に取り組んでいる。']

元の Web ページのテキストと抽出したペルソナを比較してみます。

In [12]:
for web_page, persona in zip(web_pages, personas):
    pp.pprint({"web_page_ja": web_page[:200], "persona": persona})

{'persona': 'オープンソースハードウェアとソフトウェアに関心を持つ天文愛好家であり、Arduinoやスマートフォンを用いた技術的なプロジェクトを推進するエンジニア。',
 'web_page_ja': 'This project is solving the Asteroid Watchers challenge. Description\n'
                'AROs are essentially created by combining a telescope with a smartphone. If the '
                'telescope has drive motors can be controlled via a '}
{'persona': '地質学者であり、特に砂や鉱物の成り立ちや成熟度に関心を持つ研究者。彼らは自然環境の変遷や地球の歴史を理解するために、砂のサンプルを収集し分析することを楽しむ。',
 'web_page_ja': 'Life of a Sand Grain\n'
                'THE LIFE OF A SAND GRAIN by Carl Bowser (Sept. 2018)\n'
                'They surround you almost anywhere you are in Arizona. They cling to your shoes, '
                'they end up in pockets and pant cuffs, they pr'}
{'persona': 'IoTおよびAI技術の専門家であり、データ分析やシステム設計に関心を持つエンジニアまたは研究者。',
 'web_page_ja': 'The internet of things, a system of interrelated computing devices and machines '
                'that can transfer data over a network without human interaction, has been used to '
            

Text-to-Persona 手法によって、FineWeb-Edu データセットのテキストからペルソナを抽出することが出来ました。

なお、Web ページからペルソナを抽出することは、一種の合成データ生成と考えることが出来ます。この場合は、Webページというインスタンスからペルソナという合成データを生成していますので、Text-to-Persona 手法は、**インスタンス駆動の合成データ生成**と捉えることが可能です。

#### Persona-to-Persona

Text-to-Persona だけでは抽出することが難しいペルソナがあります(例えば子供など)。そのようなペルソナをカバーするために、抽出したペルソナからさらに関連するペルソナを生成します。この手法を Persona-to-Persona と呼びます。以下のようなプロンプトを実行します。

In [13]:
def persona_to_personas(llm: LanguageModel, persona: str) -> list[str]:
    SYSTEM_PROMPT = """\
You are an AI assistant expert in finding relationships between people. Answer directly with the the new related persona definition, don't enumerate them.
"""

    USER_PROMPT = """\
Who is in close relationship with the given persona? Write just 3, each one in a different line:
{persona}

Note:
1. Your response should always start with "ペルソナ:".
2. Granularity of persona description should be similar to the input persona.
3. The output persona should be fully described without context of the input persona.
4. 日本語で回答してください。
"""

    persona = llm([[
        # {"role": "system", "content": SYSTEM_PROMPT},  # if llm supports system prompt
        # {"role": "user", "content": USER_PROMPT.format(persona=persona)},
        {"role": "user", "content": SYSTEM_PROMPT + "\n\n" + USER_PROMPT.format(persona=persona)},  # if llm does not support system prompt
    ]])[0]
    logger.debug(f"persona: {persona}")

    return [re.sub(r"^ペルソナ[:：]", "", p.strip()).strip() for p in persona.split("\n") if p.strip()][-3:]

In [14]:
new_personas_list = [persona_to_personas(llm, persona) for persona in personas[:2]]
new_personas_list

persona: ペルソナ: オープンソースハードウェアとソフトウェアを利用して天文観測を行う熱心な天文学者であり、データ解析のスキルを駆使して星の動きを追跡する研究者。ArduinoやRaspberry Piを用いて自作の観測機器を開発し、オンラインコミュニティでプロジェクトを共有することに情熱を注いでいる。

ペルソナ: 天文学とエンジニアリングの交差点に興味を持ち、大学で物理学を専攻しつつ、開発したアプリケーションで天体の位置情報を提供するアプリ開発者。プログラミング言語やデータベース管理に精通し、オープンソースプロジェクトに積極的に参加している。

ペルソナ: DIY技術や電子工作に情熱を持つエンジニアであり、天文に関するワークショップを開催し、参加者にArduinoを使った星座観察キットの制作を指導する教育者。コミュニティの中で技術を共有し、若い世代の科学への興味を引き出すことを目指している。
persona: ペルソナ: 地質学者であり、特に火山活動や地形変化に関心を持つ研究者。彼らは火山の噴火履歴やその影響を調査するために、現地調査やサンプル収集を行い、地球の動的な変化を理解することに情熱を注ぐ。

地質学者であり、古生物学にも興味を持ち、化石の研究を通じて地球の生物の進化を探求する研究者。彼らは地層の解析を行い、過去の生態系や気候変動の影響を解明することを目指す。

環境科学者であり、特に土壌の性質やその保全に焦点を当てている研究者。彼らは土壌の質とその健康が生態系に与える影響を研究し、持続可能な農業や土地利用の方法を提案する。


[['オープンソースハードウェアとソフトウェアを利用して天文観測を行う熱心な天文学者であり、データ解析のスキルを駆使して星の動きを追跡する研究者。ArduinoやRaspberry Piを用いて自作の観測機器を開発し、オンラインコミュニティでプロジェクトを共有することに情熱を注いでいる。',
  '天文学とエンジニアリングの交差点に興味を持ち、大学で物理学を専攻しつつ、開発したアプリケーションで天体の位置情報を提供するアプリ開発者。プログラミング言語やデータベース管理に精通し、オープンソースプロジェクトに積極的に参加している。',
  'DIY技術や電子工作に情熱を持つエンジニアであり、天文に関するワークショップを開催し、参加者にArduinoを使った星座観察キットの制作を指導する教育者。コミュニティの中で技術を共有し、若い世代の科学への興味を引き出すことを目指している。'],
 ['地質学者であり、特に火山活動や地形変化に関心を持つ研究者。彼らは火山の噴火履歴やその影響を調査するために、現地調査やサンプル収集を行い、地球の動的な変化を理解することに情熱を注ぐ。',
  '地質学者であり、古生物学にも興味を持ち、化石の研究を通じて地球の生物の進化を探求する研究者。彼らは地層の解析を行い、過去の生態系や気候変動の影響を解明することを目指す。',
  '環境科学者であり、特に土壌の性質やその保全に焦点を当てている研究者。彼らは土壌の質とその健康が生態系に与える影響を研究し、持続可能な農業や土地利用の方法を提案する。']]

In [15]:
for org_persona, new_personas in zip(personas, new_personas_list):
    pp.pprint({"org_persona": org_persona, "new_personas": new_personas})

{'new_personas': ['オープンソースハードウェアとソフトウェアを利用して天文観測を行う熱心な天文学者であり、データ解析のスキルを駆使して星の動きを追跡する研究者。ArduinoやRaspberry '
                  'Piを用いて自作の観測機器を開発し、オンラインコミュニティでプロジェクトを共有することに情熱を注いでいる。',
                  '天文学とエンジニアリングの交差点に興味を持ち、大学で物理学を専攻しつつ、開発したアプリケーションで天体の位置情報を提供するアプリ開発者。プログラミング言語やデータベース管理に精通し、オープンソースプロジェクトに積極的に参加している。',
                  'DIY技術や電子工作に情熱を持つエンジニアであり、天文に関するワークショップを開催し、参加者にArduinoを使った星座観察キットの制作を指導する教育者。コミュニティの中で技術を共有し、若い世代の科学への興味を引き出すことを目指している。'],
 'org_persona': 'オープンソースハードウェアとソフトウェアに関心を持つ天文愛好家であり、Arduinoやスマートフォンを用いた技術的なプロジェクトを推進するエンジニア。'}
{'new_personas': ['地質学者であり、特に火山活動や地形変化に関心を持つ研究者。彼らは火山の噴火履歴やその影響を調査するために、現地調査やサンプル収集を行い、地球の動的な変化を理解することに情熱を注ぐ。',
                  '地質学者であり、古生物学にも興味を持ち、化石の研究を通じて地球の生物の進化を探求する研究者。彼らは地層の解析を行い、過去の生態系や気候変動の影響を解明することを目指す。',
                  '環境科学者であり、特に土壌の性質やその保全に焦点を当てている研究者。彼らは土壌の質とその健康が生態系に与える影響を研究し、持続可能な農業や土地利用の方法を提案する。'],
 'org_persona': '地質学者であり、特に砂や鉱物の成り立ちや成熟度に関心を持つ研究者。彼らは自然環境の変遷や地球の歴史を理解するために、砂のサンプルを収集し分析することを楽しむ。'}


もうすこしプロンプトを工夫する必要があるかもしれませんが、与えられたペルソナと密接に関連するペルソナを生成することが出来ます。

以上で、Web ページのテキストからペルソナを抽出する方法として以下の2つの手法を紹介しました。

- Text-to-Persona
- Persona-to-Persona

### ペルソナを使って合成データを生成

ここからは、先ほど抽出したペルソナを使って合成データを生成します。今回は例として以下のような合成データを生成します。
- インストラクション
- 知識豊富なテキスト
- 数学問題

#### インストラクションの合成

まず、合成データ例として、ペルソナからインストラクション(ユーザが言語モデルに入力するプロンプト)データを合成します。

In [16]:
def generate_instruction(llm: LanguageModel, persona: str) -> dict[str, str]:
    SYSTEM_PROMPT = "You are an AI assistant expert at simulating user interactions."

    USER_PROMPT = """\
Generate a prompt the persona below might ask to an AI assistant:

{persona}

Note:
1. Your response should always start with "プロンプト:".
2. 簡潔に日本語で回答してください。
"""

    instruction = llm([[
        # {"role": "system", "content": SYSTEM_PROMPT},  # if llm supports system prompt
        # {"role": "user", "content": USER_PROMPT.format(persona=persona)},
        {"role": "user", "content": SYSTEM_PROMPT + "\n\n" + USER_PROMPT.format(persona=persona)},  # if llm does not support system prompt
    ]])[0]
    logger.debug(f"instruction: {instruction}")

    return re.sub(r"^プロンプト[:：]", "", instruction).strip()

In [17]:
instructions = [generate_instruction(llm, p) for p in personas]
instructions

instruction: プロンプト: Arduinoを使って星空観測用のセンサーを作りたいのですが、どんな部品を揃えれば良いですか？また、簡単なプロジェクト例があれば教えてください。
instruction: プロンプト: 砂の成熟度を評価するための最新の分析手法について教えてください。また、特定の鉱物が砂の成り立ちに与える影響についても知りたいです。
instruction: プロンプト: IoTデバイスから収集したデータを効果的に分析するための最適なアルゴリズムやツールについて教えてください。
instruction: プロンプト: 古代エジプトの宗教儀式に関する最新の研究成果を教えてください。また、どのようにしてこれらの知識を文化遺産の保護に活かせるか提案してください。
instruction: プロンプト: キリスト教とアジアの宗教との文化的相互作用について、具体的な事例を挙げて分析してください。
instruction: プロンプト: 回路設計における抵抗器の温度特性について教えてください。また、温度変化が回路に与える影響についても知りたいです。
instruction: プロンプト: 労働者階級が社会主義社会の建設において果たすべき具体的な役割について、どのような理論や歴史的事例がありますか？
instruction: プロンプト: 伝統的な印刷メディアがデジタル時代にどのように進化しているか、具体的な事例を挙げて教えてください。また、その効果的な活用方法についてもアドバイスをお願いします。
instruction: プロンプト: 最近の航空宇宙産業における電子機器の進化について、特にAIとセンサー技術の役割について教えてください。
instruction: プロンプト: 人工腎臓技術の最新の研究成果について教えてください。特に、患者の生活の質にどのように寄与しているか知りたいです。


['Arduinoを使って星空観測用のセンサーを作りたいのですが、どんな部品を揃えれば良いですか？また、簡単なプロジェクト例があれば教えてください。',
 '砂の成熟度を評価するための最新の分析手法について教えてください。また、特定の鉱物が砂の成り立ちに与える影響についても知りたいです。',
 'IoTデバイスから収集したデータを効果的に分析するための最適なアルゴリズムやツールについて教えてください。',
 '古代エジプトの宗教儀式に関する最新の研究成果を教えてください。また、どのようにしてこれらの知識を文化遺産の保護に活かせるか提案してください。',
 'キリスト教とアジアの宗教との文化的相互作用について、具体的な事例を挙げて分析してください。',
 '回路設計における抵抗器の温度特性について教えてください。また、温度変化が回路に与える影響についても知りたいです。',
 '労働者階級が社会主義社会の建設において果たすべき具体的な役割について、どのような理論や歴史的事例がありますか？',
 '伝統的な印刷メディアがデジタル時代にどのように進化しているか、具体的な事例を挙げて教えてください。また、その効果的な活用方法についてもアドバイスをお願いします。',
 '最近の航空宇宙産業における電子機器の進化について、特にAIとセンサー技術の役割について教えてください。',
 '人工腎臓技術の最新の研究成果について教えてください。特に、患者の生活の質にどのように寄与しているか知りたいです。']

抽出元の Web ページ、ペルソナ、インストラクションをまとめて確認してみましょう。

In [18]:
for w, p, i in zip(web_pages, personas, instructions):
    pp.pprint({"web_page": w[:100], "persona": p, "instruction": i})

{'instruction': 'Arduinoを使って星空観測用のセンサーを作りたいのですが、どんな部品を揃えれば良いですか？また、簡単なプロジェクト例があれば教えてください。',
 'persona': 'オープンソースハードウェアとソフトウェアに関心を持つ天文愛好家であり、Arduinoやスマートフォンを用いた技術的なプロジェクトを推進するエンジニア。',
 'web_page': 'This project is solving the Asteroid Watchers challenge. Description\n'
             'AROs are essentially created by'}
{'instruction': '砂の成熟度を評価するための最新の分析手法について教えてください。また、特定の鉱物が砂の成り立ちに与える影響についても知りたいです。',
 'persona': '地質学者であり、特に砂や鉱物の成り立ちや成熟度に関心を持つ研究者。彼らは自然環境の変遷や地球の歴史を理解するために、砂のサンプルを収集し分析することを楽しむ。',
 'web_page': 'Life of a Sand Grain\n'
             'THE LIFE OF A SAND GRAIN by Carl Bowser (Sept. 2018)\n'
             'They surround you almost a'}
{'instruction': 'IoTデバイスから収集したデータを効果的に分析するための最適なアルゴリズムやツールについて教えてください。',
 'persona': 'IoTおよびAI技術の専門家であり、データ分析やシステム設計に関心を持つエンジニアまたは研究者。',
 'web_page': 'The internet of things, a system of interrelated computing devices and machines that '
             'can transfer da'}
{'instruction': '古代エジプトの宗教儀式に関する最新の研究成果を教えてください。また、どのようにしてこれらの知識を文化遺産の保護に活かせるか提案して

ペルソナに該当する人物が、言語モデルに入力しそうなインストラクションを生成することが出来ました。

先ほどの図を再掲します。今回行った処理は、この図のように、`web_page` から `persona` を抽出し(`Compress`)、`persona` から `instruction` という合成データを生成した(`Decompress`)、ということになります。

<img src="https://github.com/user-attachments/assets/37ce038a-7702-4398-838d-8c504ac1da07" width="800px">

もちろん `web_page` から `instruction` を直接生成することも可能ですが、一旦 `persona` に変換することで、トークン数を大幅に削減しつつ、元のテキストの情報量を残したまま多様な合成データが生成できるというのが Persona-Hub の特徴です。

#### 知識豊富なテキストの合成

次に、同様の手法でペルソナから知識豊富なテキストを合成します。

Quoraは、ユーザがさまざまなトピックについて質問したり、回答を提供したりできる人気の Q&A サイトです。Quoraの記事は、様々な分野の専門家を含む知識豊富な人物によって書かれることが多く、質が高くよく調査された有益なコンテンツが確保されています。このアプローチによって、有益で知識豊富なコンテンツを得ることが出来ます。([Persona-Hub テクニカルレポート](https://arxiv.org/abs/2406.20094)より)

In [19]:
def generate_quora_post(llm: LanguageModel, persona: str) -> str:
    SYSTEM_PROMPT = "You are an AI assistant specialized in writing posts for social media."

    USER_PROMPT = """\
Write a Quora post in the language, style, and personality of the following persona:

{persona}

Note:
1. Your response should always start with "記事:".
2. 簡潔に日本語で回答してください。
"""

    post = llm([[
        # {"role": "system", "content": SYSTEM_PROMPT},  # if llm supports system prompt
        # {"role": "user", "content": USER_PROMPT.format(persona=persona)},
        {"role": "user", "content": SYSTEM_PROMPT + "\n\n" + USER_PROMPT.format(persona=persona)},  # if llm does not support system prompt
    ]])[0]
    logger.debug(f"post: {post}")

    return re.sub(r"^記事[:：]", "", post).strip()

In [20]:
posts = [generate_quora_post(llm, p) for p in personas[:2]]
posts

post: 記事:  
オープンソースハードウェアとソフトウェアに魅了されている天文愛好家の皆さん、こんにちは！最近、Arduinoを使って自作の天体観測装置を作成するプロジェクトに取り組んでいます。スマートフォンと連携させることで、リアルタイムで星の位置を追跡し、データを記録することができるんです。

このプロジェクトの魅力は、単に自分の天文観測を楽しむだけでなく、他の愛好者たちと知識を共有できる点です。オープンソースのコミュニティは、本当に素晴らしいですし、誰でも参加できるのが嬉しいですね。

もし皆さんも、自分だけの天文プロジェクトを立ち上げてみたいと思っているなら、Arduinoやスマートフォンを活用してみてはいかがでしょうか？アイデアや技術的なサポートが必要であれば、ぜひコメントしてください。一緒に学んでいきましょう！🌌🔭✨
post: 記事:  
こんにちは！地質学に興味がある皆さん、今日は砂や鉱物の成り立ち、そして成熟度についてお話ししたいと思います。砂は一見単純なものに見えますが、実は地球の歴史を語る貴重な手がかりです。

砂のサンプルを収集し、分析することで、私たちは過去の自然環境の変遷を理解できます。例えば、ある地域の砂がどのように形成されたのかを知ることで、その地域の気候や地形の変化を推測することができるのです。

砂の成熟度も重要な指標です。成熟した砂は、長い時間をかけて風や水の影響を受け、粒子が磨かれ、均一な大きさになります。これに対し、未成熟な砂は多様な粒子サイズを持ち、エッジが鋭い状態です。この違いから、砂がどのように移動し、変化してきたのかを読み解くことができます。

私たちの地球は常に変化しており、その証拠は砂の中に詰まっています。これからも、砂を通じて地球の物語を探求し続けたいと思います。興味のある方は、ぜひ一緒に研究をしましょう！


['オープンソースハードウェアとソフトウェアに魅了されている天文愛好家の皆さん、こんにちは！最近、Arduinoを使って自作の天体観測装置を作成するプロジェクトに取り組んでいます。スマートフォンと連携させることで、リアルタイムで星の位置を追跡し、データを記録することができるんです。\n\nこのプロジェクトの魅力は、単に自分の天文観測を楽しむだけでなく、他の愛好者たちと知識を共有できる点です。オープンソースのコミュニティは、本当に素晴らしいですし、誰でも参加できるのが嬉しいですね。\n\nもし皆さんも、自分だけの天文プロジェクトを立ち上げてみたいと思っているなら、Arduinoやスマートフォンを活用してみてはいかがでしょうか？アイデアや技術的なサポートが必要であれば、ぜひコメントしてください。一緒に学んでいきましょう！🌌🔭✨',
 'こんにちは！地質学に興味がある皆さん、今日は砂や鉱物の成り立ち、そして成熟度についてお話ししたいと思います。砂は一見単純なものに見えますが、実は地球の歴史を語る貴重な手がかりです。\n\n砂のサンプルを収集し、分析することで、私たちは過去の自然環境の変遷を理解できます。例えば、ある地域の砂がどのように形成されたのかを知ることで、その地域の気候や地形の変化を推測することができるのです。\n\n砂の成熟度も重要な指標です。成熟した砂は、長い時間をかけて風や水の影響を受け、粒子が磨かれ、均一な大きさになります。これに対し、未成熟な砂は多様な粒子サイズを持ち、エッジが鋭い状態です。この違いから、砂がどのように移動し、変化してきたのかを読み解くことができます。\n\n私たちの地球は常に変化しており、その証拠は砂の中に詰まっています。これからも、砂を通じて地球の物語を探求し続けたいと思います。興味のある方は、ぜひ一緒に研究をしましょう！']

In [21]:
for web_page, persona, post in zip(web_pages, personas, posts):
    pp.pprint({"web_page": web_page[:100], "persona": persona, "post": post[:100]})

{'persona': 'オープンソースハードウェアとソフトウェアに関心を持つ天文愛好家であり、Arduinoやスマートフォンを用いた技術的なプロジェクトを推進するエンジニア。',
 'post': 'オープンソースハードウェアとソフトウェアに魅了されている天文愛好家の皆さん、こんにちは！最近、Arduinoを使って自作の天体観測装置を作成するプロジェクトに取り組んでいます。スマートフォンと連携させ',
 'web_page': 'This project is solving the Asteroid Watchers challenge. Description\n'
             'AROs are essentially created by'}
{'persona': '地質学者であり、特に砂や鉱物の成り立ちや成熟度に関心を持つ研究者。彼らは自然環境の変遷や地球の歴史を理解するために、砂のサンプルを収集し分析することを楽しむ。',
 'post': 'こんにちは！地質学に興味がある皆さん、今日は砂や鉱物の成り立ち、そして成熟度についてお話ししたいと思います。砂は一見単純なものに見えますが、実は地球の歴史を語る貴重な手がかりです。\n'
         '\n'
         '砂のサンプルを収',
 'web_page': 'Life of a Sand Grain\n'
             'THE LIFE OF A SAND GRAIN by Carl Bowser (Sept. 2018)\n'
             'They surround you almost a'}


#### 数学問題の合成

最後に、ペルソナを使って数学の問題を合成します。数学問題については、後ほどさらに詳細なプロンプトを作成します。

In [22]:
def generate_math_problem(llm: LanguageModel, persona: str) -> str:
    SYSTEM_PROMPT = "You are an AI assistant specialized in creating diverse but specific math problems. Just answer with your problem."

    USER_PROMPT = """\
Create a challenging math problem with the following persona:

{persona}

Note:
1. Your response should always start with "問題:".
2. 簡潔に日本語で回答してください。
"""

    problem = llm([[
        # {"role": "system", "content": SYSTEM_PROMPT},  # if llm supports system prompt
        # {"role": "user", "content": USER_PROMPT.format(persona=persona)},
        {"role": "user", "content": SYSTEM_PROMPT + "\n\n" + USER_PROMPT.format(persona=persona)},  # if llm does not support system prompt
    ]])[0]
    logger.debug(f"problem: {problem}")

    return re.sub(r"^問題[:：]", "", problem).strip()

In [23]:
problems = [generate_math_problem(llm, p) for p in personas[:2]]
problems

problem: 問題: ある天文愛好家がArduinoを使って、星の位置を測定する装置を作成しました。この装置は、毎日午後8時に特定の星の高度を測定し、そのデータをスマートフォンに保存します。1ヶ月間（30日間）で、彼は合計450回のデータを収集しました。もし毎日のデータ収集回数が一定で、最初の10日間で集めたデータが全体の40%であった場合、残りの20日間で集めたデータの回数はいくつでしょうか？
problem: 問題: 地質学者の佐藤は、異なる地域から集めた5種類の砂のサンプルを分析しています。各サンプルの粒子サイズの平均値は次の通りです: サンプルAは0.25mm、サンプルBは0.15mm、サンプルCは0.35mm、サンプルDは0.20mm、サンプルEは0.30mmです。これらの平均粒子サイズの中央値を求めなさい。


['ある天文愛好家がArduinoを使って、星の位置を測定する装置を作成しました。この装置は、毎日午後8時に特定の星の高度を測定し、そのデータをスマートフォンに保存します。1ヶ月間（30日間）で、彼は合計450回のデータを収集しました。もし毎日のデータ収集回数が一定で、最初の10日間で集めたデータが全体の40%であった場合、残りの20日間で集めたデータの回数はいくつでしょうか？',
 '地質学者の佐藤は、異なる地域から集めた5種類の砂のサンプルを分析しています。各サンプルの粒子サイズの平均値は次の通りです: サンプルAは0.25mm、サンプルBは0.15mm、サンプルCは0.35mm、サンプルDは0.20mm、サンプルEは0.30mmです。これらの平均粒子サイズの中央値を求めなさい。']

In [24]:
for web_page, persona, problem in zip(web_pages, personas, problems):
    pp.pprint({"web_page": web_page[:100], "persona": persona, "problem": problem[:100]})

{'persona': 'オープンソースハードウェアとソフトウェアに関心を持つ天文愛好家であり、Arduinoやスマートフォンを用いた技術的なプロジェクトを推進するエンジニア。',
 'problem': 'ある天文愛好家がArduinoを使って、星の位置を測定する装置を作成しました。この装置は、毎日午後8時に特定の星の高度を測定し、そのデータをスマートフォンに保存します。1ヶ月間（30日間）で、彼は合計',
 'web_page': 'This project is solving the Asteroid Watchers challenge. Description\n'
             'AROs are essentially created by'}
{'persona': '地質学者であり、特に砂や鉱物の成り立ちや成熟度に関心を持つ研究者。彼らは自然環境の変遷や地球の歴史を理解するために、砂のサンプルを収集し分析することを楽しむ。',
 'problem': '地質学者の佐藤は、異なる地域から集めた5種類の砂のサンプルを分析しています。各サンプルの粒子サイズの平均値は次の通りです: '
            'サンプルAは0.25mm、サンプルBは0.15mm、サンプルCは0.35mm',
 'web_page': 'Life of a Sand Grain\n'
             'THE LIFE OF A SAND GRAIN by Carl Bowser (Sept. 2018)\n'
             'They surround you almost a'}


以上で、抽出したペルソナを使って以下の3つの合成データを生成しました。

- インストラクション
- 知識豊富なテキスト
- 数学問題

### Persona-Hub 手法のまとめ

ここまで Persona-Hub 手法を紹介しました。以下にまとめます。

<img src="https://github.com/user-attachments/assets/344b011d-b9f8-4ac3-a79a-198e3862b3cf" width="800px">

- 既存の合成データ⽣成⼿法(インスタンス駆動・キーポイント駆動)では合成データの多様さをスケールアップすることが困難
- ⼤規模なペルソナデータセットを作成し、それを使ってスケーラブルな合成データを⽣成する(ペルソナ駆動)
- Web ページからペルソナを抽出
  - Text-to-Persona
  - Persona-to-Persona
- ペルソナを使って合成データを生成
  - インストラクション
  - 知識豊富なテキスト
  - 数学問題
- テクニカルレポート: https://arxiv.org/abs/2406.20094

## FinePersonas による事後学習データ合成

### FinePersonas とは

ここからは大規模なペルソナデータセットである [FinePersonas](https://huggingface.co/datasets/argilla/FinePersonas-v0.1) について説明します。

<img src="https://cdn-uploads.huggingface.co/production/uploads/6435d564a4bd75c62cc03701/5wTHwgijTUKFI5B-N7gEg.png" width="600px">

[FinePersonas](https://huggingface.co/datasets/argilla/FinePersonas-v0.1) は Argilla が2024年9月にリリースされた合成テキスト生成のためのペルソナデータセットです。以下のような特徴があります。

- 2100万人の詳細なペルソナのオープンデータセット
- 合成データの豊富さ・多様性・特異性を高めることが可能
- [Persona-Hub](https://arxiv.org/abs/2406.20094) と同じレシピに従い、[FineWeb-Edu](https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu) (教育関連Webページのデータセット)から2100万件のペルソナを抽出
- ライセンス: Llama 3.1 Community License Agreement

### FinePersonas データセットの読み込み

実際のペルソナデータを読み込んで、数学の問題と解答を合成します。[FinePersonas](https://huggingface.co/datasets/argilla/FinePersonas-v0.1) データセットを利用します。

FinePersonas データセットは巨大なデータセットですので、本ハンズオンでは一部分だけをダウンロードして利用します。読み込む際に `streaming=True` を必ずつけてください。

In [25]:
dataset = load_dataset("argilla/FinePersonas-v0.1", split="train", cache_dir="cache", streaming=True)
dataset

IterableDataset({
    features: ['id', 'persona', 'labels'],
    num_shards: 12
})

今回は先頭の100件を使って生成します。

In [26]:
personas = [data["persona"] for data in dataset.take(100)]
pp.pprint(personas[:10])

['A professional R programmer or researcher, likely a data analyst or statistician, familiar with '
 'the intricacies of the R language and its debugging tools.',
 'A mental health professional, likely a licensed therapist or psychologist, with expertise in '
 'anxiety disorders and cognitive-behavioral therapy, whose work involves diagnosing and treating '
 'patients with various types of phobias, including specific phobia, social phobia, and '
 'agoraphobia.',
 'A space roboticist or aerospace engineer at a research institution focused on robotic space '
 'exploration.',
 'A Hebrew language scholar or instructor with a focus on biblical Hebrew and Jewish studies.',
 'A pediatrician or healthcare professional focused on educating parents about early childhood '
 'health, or a parent-to-be who is interested in learning about vaccinations for their unborn or '
 'newborn child.',
 'A Montessori elementary school teacher or an education administrator responsible for curriculum '
 'develop

### マルチターン対話データの合成

ここからは事後学習のためのマルチターン対話データを生成します。具体的には、

- 質問1 (`Q1`): ユーザが質問をする (この部分を Persona-Hub 手法で生成)
- 解答1 (`A1`): アシスタントが解答する
- 質問2 (`Q2`): ユーザが追加質問をする
- 解答2 (`A2`): アシスタントが追加質問に解答する

という4つの対話データを生成します。このうち `Q1` の生成に Persona-Hub 手法を適用します。それ以降の対話データは、若干の工夫をしますが基本的に言語モデルの生成に任せます。

#### Q1 の生成

まず、Persona-Hub 手法を使って `Q1` の生成を行います。

[元論文のコード](https://github.com/tencent-ailab/persona-hub/blob/main/code/prompt_templates.py)では、数学の問題を生成するプロンプトとして以下が紹介されています。

```python
math_template = '''Create a math problem related to the following persona:

{persona}

Note:

1. The math problem should be challenging and involve advanced mathematical skills and knowledge. Only top talents can solve it correctly.
2. You should make full use of the persona description to create the math problem to ensure that the math problem is unique and specific to the persona.
3. Your response should always start with "Math problem:". Your response should not include a solution to the created math problem.
4. Your created math problem should include no more than 2 sub-problems.
'''
```

これを参考に、プロンプトの一部を修正/パラメータ化して使います。修正点は以下の通りです。

- 一部の単語をパラメータ化
  - タスクの種類: `task` (e.g. "math", "reasoning", "coding")
  - トピックの種類: `topic` (e.g. "persona", "topic")
  - トピックの内容: `item` (e.g. "SF作家", "経営コンサルタント")
  - 対象者: `target` (e.g. "grade school student", "graduate student")
- 難易度を調整
  - 元のプロンプトはかなり難易度の高い問題を生成するような表現(`challenging`, `advanced`, `top talents`)となっていたため、難易度を下げるように表現を修正(`simple`, `basic`, `average`)
- 小問を生成しないよう修正
  - 小問を生成すると対話が複雑になりすぎるため、小問を生成しないように修正(`sub-problems`の行を削除)。代わりにマルチターンで追加質問をするようにする。
- 日本語を出力
  - 元のプロンプトは英語を出力するようになっているため、日本語を出力するように促すプロンプトを追加

上記の修正をしたプロンプトを OpenAI messages 形式で出力する関数を作成します。

また、利用する言語モデルで推奨されているシステムプロンプトがある場合は、それを指定します(コメントアウトされた `{"role": "system", "content": "..."}` の部分)。

In [27]:
def get_q1_prompt(task: str, topic: str, item: str, target: str) -> list[dict[str, str]]:
    Q1_PROMPT_TEMPLATE = """Create a {task} problem related to the following {topic}:

{item}

Note:

1. The {task} problem should be simple and involve basic {task} skills and knowledge. Any average {target} can solve it correctly.
2. You should make full use of the {topic} description to create the {task} problem to ensure that the {task} problem is unique and specific to the {topic}.
3. Your response should always start with "問題:". Your response should not include a solution to the created {task} problem.
4. 簡潔に日本語で回答してください。
"""
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": Q1_PROMPT_TEMPLATE.format(task=task, topic=topic, item=item, target=target)},
    ]

実行して動作を確認します。

In [28]:
q1_prompt = get_q1_prompt("math", "persona", personas[0], "grade school student")
pp.pprint(q1_prompt)

[{'content': 'Create a math problem related to the following persona:\n'
             '\n'
             'A professional R programmer or researcher, likely a data analyst or statistician, '
             'familiar with the intricacies of the R language and its debugging tools.\n'
             '\n'
             'Note:\n'
             '\n'
             '1. The math problem should be simple and involve basic math skills and knowledge. '
             'Any average grade school student can solve it correctly.\n'
             '2. You should make full use of the persona description to create the math problem to '
             'ensure that the math problem is unique and specific to the persona.\n'
             '3. Your response should always start with "問題:". Your response should not include a '
             'solution to the created math problem.\n'
             '4. 簡潔に日本語で回答してください。\n',
  'role': 'user'}]


言語モデルで推論して Q1 を生成します。

In [29]:
q1s = llm([get_q1_prompt("math", "persona", personas[0], "grade school student")])
pp.pprint(q1s)

['問題: R言語を使ってデータを分析している研究者がいます。彼は、データセットの中から特定の5つの数値を抽出しました。これらの数値はそれぞれ3, 7, 2, 5, '
 '8です。これらの数値の合計はいくつですか？']


引数を変更して難易度の高い問題を生成します。

In [30]:
q1s = llm([get_q1_prompt("advanced math", "persona", personas[0], "graduate student")])
pp.pprint(q1s)

['問題:  \n'
 'あるデータ分析者が、R言語を用いてあるデータセットの回帰分析を行っています。データセットには、独立変数 \\(X\\) と従属変数 \\(Y\\) '
 'のペアが含まれています。回帰モデルの式が次のように与えられています。\n'
 '\n'
 '\\[\n'
 'Y = \\beta_0 + \\beta_1 X + \\epsilon\n'
 '\\]\n'
 '\n'
 'ここで、\\(\\beta_0\\) は切片、\\(\\beta_1\\) は傾き、\\(\\epsilon\\) は誤差項です。このデータ分析者は、\\(\\beta_1\\) '
 'の値が0.5であると仮定し、切片 \\(\\beta_0\\) は未知であるとします。\n'
 '\n'
 'データセットの中で、\\(X\\) の平均が10、標準偏差が2、\\(Y\\) の平均が20、標準偏差が3であることがわかっています。このとき、\\(\\beta_0\\) '
 'の推定値を求めるために必要な数式を導出し、推定値を計算してください。ただし、相関係数 \\(r\\) は0.8とします。']


問題さなさそうです。

次に、`問題:` など余分な部分をルールベース(正規表現)で削除します。言語モデルによっては、解答やヒントを書いてしまうものがありますので、もしそれらが出力されるのであればここで削除しておきます。

**Note: この関数が複雑になり過ぎる場合は、プロンプトの見直しや言語モデルの変更を検討した方が良いかもしれません。**

In [31]:
def filter_q1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^問題[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^Problem[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"\n答え[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[解回]答[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[Aa]nswer[：:].*", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

実行して動作を確認します。

In [32]:
filter_q1(q1s[0])

'あるデータ分析者が、R言語を用いてあるデータセットの回帰分析を行っています。データセットには、独立変数 \\(X\\) と従属変数 \\(Y\\) のペアが含まれています。回帰モデルの式が次のように与えられています。\n\n\\[\nY = \\beta_0 + \\beta_1 X + \\epsilon\n\\]\n\nここで、\\(\\beta_0\\) は切片、\\(\\beta_1\\) は傾き、\\(\\epsilon\\) は誤差項です。このデータ分析者は、\\(\\beta_1\\) の値が0.5であると仮定し、切片 \\(\\beta_0\\) は未知であるとします。\n\nデータセットの中で、\\(X\\) の平均が10、標準偏差が2、\\(Y\\) の平均が20、標準偏差が3であることがわかっています。このとき、\\(\\beta_0\\) の推定値を求めるために必要な数式を導出し、推定値を計算してください。ただし、相関係数 \\(r\\) は0.8とします。'

問題なさそうです。

作成した関数を組み合わせて、バッチ処理で `Q1` を生成する関数を作成します。

In [33]:
def generate_q1(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[str]:
    return [
        filter_q1(q1)
        for q1 in llm([
            get_q1_prompt(task, topic, item, target)
            for task, topic, item, target in zip(tasks, topics, items, targets)
        ])
    ]

実行して動作を確認します。

In [34]:
q1s = generate_q1(
    llm,
    ["math", "advanced math"],
    ["persona", "persona"],
    personas[:2],
    ["grade school student", "graduate student"],
)
pp.pprint(q1s)

['R言語を使ってデータを分析している研究者がいます。彼は、データセットの中から特定の5つの数値を抽出しました。これらの数値はそれぞれ3, 7, 2, 5, '
 '10です。彼はこれらの数値の合計を求めたいと考えています。これらの数値の合計はいくつになりますか？',
 'ある精神健康専門家が、特定の恐怖症と社会不安障害を抱える患者の治療を行っています。彼は、10人の患者を対象にした研究を行い、その中の6人が特定の恐怖症を持ち、4人が社会不安障害を持っています。この専門家は、各患者が受ける治療セッションの平均時間を30分とし、特定の恐怖症を持つ患者には週に2回、社会不安障害を持つ患者には週に1回のセッションを提供することに決定しました。\n'
 '\n'
 '1. それぞれの患者タイプに対する合計治療時間を計算してください。\n'
 '2. 各患者タイプの治療セッションの合計回数を求めてください。\n'
 '3. 合計治療時間を合計回数で割って、各セッションの平均時間を出してください。']


問題なさそうです。ここまでで `Q1` を生成する関数の実装が完了しました。

#### A1 の生成

次に、`A1` の生成を行います。ここでは基本的に `Q1` を入力して、長さ指定(`簡潔に`)と、出力言語の指定(`日本語で`)をして言語モデルに生成させます。言語モデルによっては、表現を微調整する必要があるかもしれません。

実行の流れは `Q1` と同様ですので、ここではコードのみ記載します。

In [35]:
def get_a1_prompt(q1: str) -> list[dict]:
    A1_PROMPT_TEMPLATE = "{q1}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": A1_PROMPT_TEMPLATE.format(q1=q1)},
    ]

In [36]:
a1_prompt = get_a1_prompt(q1s[0])
pp.pprint(a1_prompt)

[{'content': 'R言語を使ってデータを分析している研究者がいます。彼は、データセットの中から特定の5つの数値を抽出しました。これらの数値はそれぞれ3, 7, 2, 5, '
             '10です。彼はこれらの数値の合計を求めたいと考えています。これらの数値の合計はいくつになりますか？\n'
             '\n'
             '簡潔に日本語で回答してください。',
  'role': 'user'}]


In [37]:
a1s = llm([get_a1_prompt(q1s[0])])
pp.pprint(a1s)

['合計は27です。']


In [38]:
a1s = llm([get_a1_prompt(q1s[1])])
pp.pprint(a1s)

['1. **合計治療時間の計算**:\n'
 '   - 特定の恐怖症を持つ患者: 6人 × 30分 × 2回/週 = 360分\n'
 '   - 社会不安障害を持つ患者: 4人 × 30分 × 1回/週 = 120分\n'
 '   - 合計治療時間 = 360分 + 120分 = 480分\n'
 '\n'
 '2. **治療セッションの合計回数**:\n'
 '   - 特定の恐怖症を持つ患者: 6人 × 2回/週 = 12回\n'
 '   - 社会不安障害を持つ患者: 4人 × 1回/週 = 4回\n'
 '   - 合計治療セッション数 = 12回 + 4回 = 16回\n'
 '\n'
 '3. **平均セッション時間の計算**:\n'
 '   - 合計治療時間 / 合計回数 = 480分 / 16回 = 30分\n'
 '\n'
 '以上の結果から、各セッションの平均時間は30分です。']


In [39]:
def filter_a1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [40]:
def generate_a1(llm: LanguageModel, q1s: list[str]) -> list[str]:
    return [filter_a1(a1) for a1 in llm([get_a1_prompt(q1) for q1 in q1s])]

In [41]:
a1s = generate_a1(llm, q1s)
pp.pprint(a1s)

['合計は27です。',
 '1. **合計治療時間の計算**:\n'
 '   - 特定の恐怖症を持つ患者: 6人 × 30分 × 2回/週 = 360分\n'
 '   - 社会不安障害を持つ患者: 4人 × 30分 × 1回/週 = 120分\n'
 '   - **合計治療時間** = 360分 + 120分 = **480分**\n'
 '\n'
 '2. **治療セッションの合計回数**:\n'
 '   - 特定の恐怖症を持つ患者: 6人 × 2回/週 = 12回\n'
 '   - 社会不安障害を持つ患者: 4人 × 1回/週 = 4回\n'
 '   - **合計回数** = 12回 + 4回 = **16回**\n'
 '\n'
 '3. **平均セッション時間の計算**:\n'
 '   - 合計治療時間: 480分\n'
 '   - 合計回数: 16回\n'
 '   - **平均時間** = 480分 ÷ 16回 = **30分** \n'
 '\n'
 '以上の結果です。']


#### Q2 の生成

次に、追加質問 `Q2` の生成を行います。

- まず `Q1` に関連する質問であることを強調します(`前述の問題をより理解するために`の部分)。これが無いと `Q2` で新たな別の質問を生成してしまうことがあります。
- 次に `問題の一部を変更したり、条件を追加しても良いです` という部分で、`Q1` の問題の一部を変更することを促します。これにより、`Q2` が `Q1` に関連する質問となる確率を高めることができます。
- 最後に答えを含まないように注意を促します(`決して答えを含めないでください`の部分)。

In [42]:
def get_q2_prompt(q1: str, a1: str) -> list[dict[str, str]]:
    Q2_PROMPT_TEMPLATE = "前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": Q2_PROMPT_TEMPLATE},
    ]

In [43]:
q2_prompt = get_q2_prompt(q1s[0], a1s[0])
pp.pprint(q2_prompt)

[{'content': 'R言語を使ってデータを分析している研究者がいます。彼は、データセットの中から特定の5つの数値を抽出しました。これらの数値はそれぞれ3, 7, 2, 5, '
             '10です。彼はこれらの数値の合計を求めたいと考えています。これらの数値の合計はいくつになりますか？',
  'role': 'user'},
 {'content': '合計は27です。', 'role': 'assistant'},
 {'content': '前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。',
  'role': 'user'}]


In [44]:
q2s = llm([get_q2_prompt(q1s[0], a1s[0])])
pp.pprint(q2s)

['これらの数値の平均を求めるには、どのように計算しますか？']


In [45]:
def filter_q2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^追加の質問[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^質問[：:]", "", content, flags=re.DOTALL)
    content = content.strip()
    return content

In [46]:
def generate_q2(llm: LanguageModel, q1s: list[str], a1s: list[str]) -> list[str]:
    return [filter_q2(q2) for q2 in llm([get_q2_prompt(q1, a1) for q1, a1 in zip(q1s, a1s)])]

In [47]:
q2s = generate_q2(llm, q1s, a1s)
pp.pprint(q2s)

['これらの数値の平均を求めるには、どのように計算しますか？', '特定の恐怖症を持つ患者には、特別なワークショップを月に1回追加で行うことにした場合、月間の合計治療時間はどのように変わりますか？']


#### A2 の生成

最後に、`A2` の生成を行います。基本的に `A1` の生成と同様ですので、ここではコードのみ記載します。

In [48]:
def get_a2_prompt(q1: str, a1: str, q2: str) -> list[dict[str, str]]:
    A2_PROMPT_TEMPLATE = "{q2}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": A2_PROMPT_TEMPLATE.format(q2=q2)},
    ]

In [49]:
a2_prompt = get_a2_prompt(q1s[0], a1s[0], q2s[0])
pp.pprint(a2_prompt)

[{'content': 'R言語を使ってデータを分析している研究者がいます。彼は、データセットの中から特定の5つの数値を抽出しました。これらの数値はそれぞれ3, 7, 2, 5, '
             '10です。彼はこれらの数値の合計を求めたいと考えています。これらの数値の合計はいくつになりますか？',
  'role': 'user'},
 {'content': '合計は27です。', 'role': 'assistant'},
 {'content': 'これらの数値の平均を求めるには、どのように計算しますか？\n\n簡潔に日本語で回答してください。', 'role': 'user'}]


In [50]:
a2s = llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])
pp.pprint(a2s)

['これらの数値の平均を求めるには、合計を数値の個数で割ります。具体的には、合計27を5で割ります。計算式は次の通りです。\n'
 '\n'
 '\\[\n'
 '\\text{平均} = \\frac{27}{5} = 5.4\n'
 '\\] \n'
 '\n'
 'したがって、平均は5.4です。',
 '特定の恐怖症を持つ患者には、月に1回の特別なワークショップが追加されます。ワークショップの時間は30分と仮定すると、6人の患者全員が参加するため、追加の治療時間は次のように計算されます。\n'
 '\n'
 '- 追加の治療時間: 6人 × 30分 = 180分\n'
 '\n'
 '元の月間治療時間は、特定の恐怖症の360分と社会不安障害の120分を合わせて480分です。\n'
 '\n'
 '月間の合計治療時間は:\n'
 '- 480分 + 180分 = **660分**\n'
 '\n'
 'したがって、月間の合計治療時間は660分に増加します。']


In [51]:
def filter_a2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [52]:
def generate_a2(llm: LanguageModel, q1s: list[str], a1s: list[str], q2s: list[str]) -> list[str]:
    return [filter_a2(a2) for a2 in llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])]

In [53]:
a2s = generate_a2(llm, q1s, a1s, q2s)
pp.pprint(a2s)

['これらの数値の平均を求めるには、合計を数値の個数で割ります。具体的には、合計27を5で割ります。計算式は次の通りです。\n'
 '\n'
 '\\[\n'
 '\\text{平均} = \\frac{27}{5} = 5.4\n'
 '\\] \n'
 '\n'
 'したがって、平均は5.4です。',
 '特定の恐怖症を持つ患者には、月に1回の特別なワークショップが追加されます。ワークショップの時間は30分と仮定すると、6人の患者全員が参加するため、追加の治療時間は次のように計算されます。\n'
 '\n'
 '- 追加の治療時間: 6人 × 30分 = 180分\n'
 '\n'
 '元の月間治療時間は、特定の恐怖症の360分と社会不安障害の120分を合わせて480分でした。\n'
 '\n'
 '**新しい月間合計治療時間**:\n'
 '480分 + 180分 = **660分**\n'
 '\n'
 'したがって、月間の合計治療時間は660分に増加します。']


ここまでの対話データの流れを確認します。

In [54]:
pp.pprint(list(zip(q1s, a1s, q2s, a2s))[0])

('R言語を使ってデータを分析している研究者がいます。彼は、データセットの中から特定の5つの数値を抽出しました。これらの数値はそれぞれ3, 7, 2, 5, '
 '10です。彼はこれらの数値の合計を求めたいと考えています。これらの数値の合計はいくつになりますか？',
 '合計は27です。',
 'これらの数値の平均を求めるには、どのように計算しますか？',
 'これらの数値の平均を求めるには、合計を数値の個数で割ります。具体的には、合計27を5で割ります。計算式は次の通りです。\n'
 '\n'
 '\\[\n'
 '\\text{平均} = \\frac{27}{5} = 5.4\n'
 '\\] \n'
 '\n'
 'したがって、平均は5.4です。')


In [55]:
pp.pprint(list(zip(q1s, a1s, q2s, a2s))[1])

('ある精神健康専門家が、特定の恐怖症と社会不安障害を抱える患者の治療を行っています。彼は、10人の患者を対象にした研究を行い、その中の6人が特定の恐怖症を持ち、4人が社会不安障害を持っています。この専門家は、各患者が受ける治療セッションの平均時間を30分とし、特定の恐怖症を持つ患者には週に2回、社会不安障害を持つ患者には週に1回のセッションを提供することに決定しました。\n'
 '\n'
 '1. それぞれの患者タイプに対する合計治療時間を計算してください。\n'
 '2. 各患者タイプの治療セッションの合計回数を求めてください。\n'
 '3. 合計治療時間を合計回数で割って、各セッションの平均時間を出してください。',
 '1. **合計治療時間の計算**:\n'
 '   - 特定の恐怖症を持つ患者: 6人 × 30分 × 2回/週 = 360分\n'
 '   - 社会不安障害を持つ患者: 4人 × 30分 × 1回/週 = 120分\n'
 '   - **合計治療時間** = 360分 + 120分 = **480分**\n'
 '\n'
 '2. **治療セッションの合計回数**:\n'
 '   - 特定の恐怖症を持つ患者: 6人 × 2回/週 = 12回\n'
 '   - 社会不安障害を持つ患者: 4人 × 1回/週 = 4回\n'
 '   - **合計回数** = 12回 + 4回 = **16回**\n'
 '\n'
 '3. **平均セッション時間の計算**:\n'
 '   - 合計治療時間: 480分\n'
 '   - 合計回数: 16回\n'
 '   - **平均時間** = 480分 ÷ 16回 = **30分** \n'
 '\n'
 '以上の結果です。',
 '特定の恐怖症を持つ患者には、特別なワークショップを月に1回追加で行うことにした場合、月間の合計治療時間はどのように変わりますか？',
 '特定の恐怖症を持つ患者には、月に1回の特別なワークショップが追加されます。ワークショップの時間は30分と仮定すると、6人の患者全員が参加するため、追加の治療時間は次のように計算されます。\n'
 '\n'
 '- 追加の治療時間: 6人 × 30分 = 180分\n'
 '\n'
 '元の月間治療時間は、特定の恐怖症の360分と

面白い問題かどうかはともかく、問題なさそうです。これで `A2` の生成も完了しました。

### マルチターン対話データの生成

これまで作成した関数を組み合わせて、マルチターンの対話データを生成する関数を作成します。結果は OpenAI messages 形式で出力します。

In [56]:
def synthesis_multi_turn_qa(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[dict[str, str | list[dict[str, str]]]]:
    q1s = generate_q1(llm, tasks, topics, items, targets)
    a1s = generate_a1(llm, q1s)
    q2s = generate_q2(llm, q1s, a1s)
    a2s = generate_a2(llm, q1s, a1s, q2s)
    return [
        {
            "messages": [
                {"role": "user", "content": q1},
                {"role": "assistant", "content": a1},
                {"role": "user", "content": q2},
                {"role": "assistant", "content": a2},
            ],
            "task": task,
            "topic": topic,
            "item": item,
            "target": target,
        }
        for task, topic, item, target, q1, a1, q2, a2 in zip(tasks, topics, items, targets, q1s, a1s, q2s, a2s)
    ]

実行して動作を確認します。

In [57]:
qas = synthesis_multi_turn_qa(
    llm,
    ["math"],
    ["persona"],
    personas[:1],
    ["grade school student"],
)
pp.pprint(qas)

[{'item': 'A professional R programmer or researcher, likely a data analyst or statistician, '
          'familiar with the intricacies of the R language and its debugging tools.',
  'messages': [{'content': 'R言語を使ってデータを分析している研究者がいます。彼は、データセットの中から特定の5つの数値を抽出しました。これらの数値はそれぞれ3、7、2、9、5です。彼はこの5つの数値の合計を求めたいと思っています。合計はいくつになりますか？',
                'role': 'user'},
               {'content': '合計は26です。', 'role': 'assistant'},
               {'content': 'この5つの数値の平均を求めると、いくつになりますか？', 'role': 'user'},
               {'content': '平均は5.2です。', 'role': 'assistant'}],
  'target': 'grade school student',
  'task': 'math',
  'topic': 'persona'}]


この関数をバッチ処理で実行し、結果を JSONL ファイルに保存する関数を作成します。`task`, `topic`, `item`, `target` は、与えられた引数からランダムに選択して、対話データを生成します。

In [58]:
def run_synthesis_multi_turn_qa(
    llm: LanguageModel,
    task_list: list[str],
    topic_list: list[str],
    item_list: list[str],
    target_list: list[str],
    num_samples: int = 1,
    batch_size: int = 1,
    output_jsonl: str = "output.jsonl",
):
    with open(output_jsonl, "a", encoding="utf-8") as f:
        for i in range(num_samples // batch_size):
            tasks = [random.choice(task_list) for _ in range(batch_size)]
            topics = [random.choice(topic_list) for _ in range(batch_size)]
            items = [random.choice(item_list) for _ in range(batch_size)]
            targets = [random.choice(target_list) for _ in range(batch_size)]
            assert len(tasks) == len(topics) == len(items) == len(targets) == batch_size
            qas = synthesis_multi_turn_qa(llm, tasks, topics, items, targets)
            for qa in qas:
                f.write(json.dumps(qa, ensure_ascii=False) + "\n")
                print(pp.pformat(qa))

ペルソナデータから対話データを生成します。

In [59]:
run_synthesis_multi_turn_qa(
    llm,
    ["math"],
    ["persona"],
    personas,
    ["grade school student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'An aerospace materials engineer focused on advanced ceramic coatings and plasma '
         'deposition techniques for high-temperature applications.',
 'messages': [{'content': 'ある航空宇宙材料エンジニアが、耐熱性のセラミックコーティングを施すために、直径1メートルの円形の金属面を持っています。この金属面にセラミックコーティングをするのに必要な時間は、1平方メートルあたり2時間です。この金属面の表面積を求め、コーティングにかかる総時間を計算してください。ただし、コーティングは表面全体に施されるものとします。',
               'role': 'user'},
              {'content': '直径1メートルの円形の金属面の半径は0.5メートルです。円の表面積は以下の式で求められます。\n'
                          '\n'
                          '\\[\n'
                          '\\text{表面積} = \\pi r^2 = \\pi (0.5)^2 = \\pi \\times 0.25 \\approx '
                          '0.785 \\, \\text{平方メートル}\n'
                          '\\]\n'
                          '\n'
                          'コーティングにかかる時間は、1平方メートルあたり2時間なので、\n'
                          '\n'
                          '\\[\n'
                          '\\text{総時間} = 0.785 \\, \\text{平方メートル} \\times 2 \\, \\text{時間/平方メートル} '
                         

難易度を上げて生成します。

In [60]:
run_synthesis_multi_turn_qa(
    llm,
    ["advanced math"],
    ["persona"],
    personas,
    ["graduate student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'An astrophysicist specializing in the study of planetary magnetism and aurora phenomena, '
         'likely with a focus on exoplanetary systems and space exploration.',
 'messages': [{'content': 'ある外惑星系において、惑星Aの磁場の強さは、地表からの距離rに対して次の式で表されるとする：  \n'
                          '\\[ B(r) = B_0 \\left( \\frac{R}{r} \\right)^3 \\]  \n'
                          'ここで、\\( B_0 \\)は惑星の中心における磁場の強さ、Rは惑星の半径、rは地表からの距離です。\n'
                          '\n'
                          '惑星Aの半径は7000 km、中心における磁場の強さは0.5 Tである。  \n'
                          'この惑星の地表から1000 kmの高さにおける磁場の強さB(1000)を求めよ。  \n'
                          'また、地表からどの高さで磁場の強さが0.1 Tになるかを求め、その高さを答えよ。',
               'role': 'user'},
              {'content': 'まず、与えられた情報を整理します。\n'
                          '\n'
                          '- 惑星の半径 \\( R = 7000 \\) km\n'
                          '- 中心における磁場の強さ \\( B_0 = 0.5 \\) T\n'
                          '\n'
                          '地表からの距離 \\( r \\) は、地表からの高さに惑星の半径を加えたものです。

同様に、論理推論の問題も生成してみましょう。論理推論の問題生成は難易度が高いため、性能の低い言語モデルではうまくいかないかもしれません。

In [61]:
run_synthesis_multi_turn_qa(
    llm,
    ["logical reasoning"],
    ["persona"],
    personas,
    ["grade school student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'A herpetologist specializing in crocodile biology, behavior, and conservation.',
 'messages': [{'content': 'ある日、爬虫類学者の田中さんは、3匹のワニを観察しています。彼はそれぞれのワニに名前を付けました。ワニAは水辺でよく見かけられ、ワニBは昼間によく日光浴をし、ワニCは夜に活動的です。田中さんは次のことを知っています：\n'
                          '\n'
                          '1. ワニAは昼間にはあまり見かけない。\n'
                          '2. ワニBは水辺にあまり近づかない。\n'
                          '3. ワニCは日中はほとんど動かない。\n'
                          '\n'
                          'これらの情報から、次の質問に答えてください。ワニAが最も活発に活動する時間帯はいつですか？',
               'role': 'user'},
              {'content': 'ワニAが最も活発に活動する時間帯は夜です。', 'role': 'assistant'},
              {'content': 'ワニDは朝に活動的で、ワニAとワニBは水辺で一緒にいることがある。ワニDの行動はワニAにどのような影響を与えると考えられますか？',
               'role': 'user'},
              {'content': 'ワニDが朝に活動的であるため、ワニAが朝の時間帯にはあまり見かけないことから、ワニDの存在はワニAの活動を抑制する可能性があります。ワニAは昼間あまり見かけないため、ワニDと一緒にいることがあるのは、主に水辺での活動時間帯が異なるためと考えられます。',
               'role': 'assistant'}],
 'target': 'grade school student',
 'task': 'logical reaso

コーディングの問題も生成してみましょう。コーディングに特化した言語モデルを使うとより良い結果が得られるかもしれません。

In [62]:
run_synthesis_multi_turn_qa(
    llm,
    ["Python coding"],
    ["persona"],
    personas,
    ["high school student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'A neuroscientist or biologist studying circadian rhythms and their impact on behavioral '
         'development.',
 'messages': [{'content': 'ある神経科学者が、サンプルデータとして24時間の間に観察された動物の行動パターンを収集しました。データは、時間帯（0から23までの整数）と、その時間帯に観察された行動の数を含むリストの形で与えられます。例えば、`[(0, '
                          '5), (1, 10), (2, 15), ..., (23, 2)]`のようになります。\n'
                          '\n'
                          'このデータを使って、次の2つの機能を持つPythonプログラムを作成してください。\n'
                          '\n'
                          '1. '
                          '`get_peak_activity(data)`関数を定義し、最も多くの行動が観察された時間帯を返します。返り値は、その時間帯と行動の数のタプル（時間帯, '
                          '行動の数）とします。\n'
                          '\n'
                          '2. `average_activity(data)`関数を定義し、全時間帯における平均行動数を計算して返します。\n'
                          '\n'
                          '例:\n'
                          '```python\n'
                          'data = [(0, 5), (1, 10), (2, 15), (3, 8), (4, 12), ..., (23, 2)]\n'
                          

以上のように、様々な分野の問題と解答のマルチターン対話データを生成することが確認できました。

## まとめ

本ハンズオンでは、言語モデルを使って合成データを作成する方法を紹介しました。以下、本ハンズオンで紹介した内容をまとめます。

- 合成データ生成の概要
- [Persona-Hub](https://arxiv.org/abs/2406.20094) 手法の解説
  - Web ページのテキストからペルソナを抽出
    - Text-to-Persona
    - Persona-to-Persona
  - ペルソナを使って合成データを生成
    - インストラクション
    - 知識豊富なテキスト
    - 数学問題
- [FinePersonas](https://huggingface.co/datasets/argilla/FinePersonas-v0.1) データセットを使ったマルチターン事後学習データの合成
  - Q1: ユーザが質問をする
  - A1: アシスタントが解答する
  - Q2: ユーザが追加質問をする
  - A2: アシスタントが追加質問に解答する

## 参考文献

- Xin Chan et al., "Scaling Synthetic Data Creation with 1,000,000,000 Personas”, arXiv preprint arXiv:2406.20094v1, 2024. https://arxiv.org/abs/2406.20094
- Guilherme Penedo et al., "The FineWeb Datasets: Decanting the Web for the Finest Text Data at Scale", arXiv preprint arXiv:2406.17557v2, 2024. https://arxiv.org/abs/2406.17557
- Hao Chen et al., "On the Diversity of Synthetic Data and its Impact on Training Large Language Models", arXiv preprint arXiv:2410.15226v2, 2024. https://arxiv.org/abs/2410.15226
- Lozhkov et al., "FineWeb-Edu: the Finest Collection of Educational Content", アクセス日: 2025-01-28, https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu
- Argilla, “FinePersonas”, アクセス日: 2025-01-28, https://huggingface.co/datasets/argilla/FinePersonas-v0.1
- Kan Hatakeyama, “大規模言語モデルTanuki-8x8Bの紹介と開発経緯など”, 9/10 松尾研LLM開発プロジェクト “Tanuki-8x8B” 開発成果報告会 Vol.1, https://www.docswell.com/s/matsuo-lab_llm/51R2L4-2024-9-10-Tanuki%E9%96%8B%E7%99%BA%E5%A0%B1%E5%91%8A%E4%BC%9A-vol1, アクセス日: 2025-01-28
- Susumu Ota, "Persona-Hub による合成データ生成", 9/24 松尾研LLM開発プロジェクト “Tanuki-8x8B” 開発成果報告会 Vol. 3, https://www.docswell.com/s/matsuo-lab_llm/ZDNGR4-2024-9-24-Tanuki%E9%96%8B%E7%99%BA%E5%A0%B1%E5%91%8A%E4%BC%9A-vol3, アクセス日: 2025-01-28
- [NEDO 採択プロジェクト] 多様な日本語能力の向上を目指した公開の基盤モデル開発, コードレポジトリ “synth_topic_multiturn.py”,アクセス日: 2025-01-28, https://github.com/matsuolab/nedo_project_code/blob/team_hatakeyama_phase2/team_hatakeyama_phase2/ota/topic-hub/synth_topic_multiturn.py


以上